## refer to: https://www.kaggle.com/raddar/paris-madness

# 1.import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections
import datetime

pd.set_option("display.max_column", 999)
# print(os.listdir("../input"))

# 2.Load the data!

In [3]:
path = '../W/input/google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/WDataFiles_Stage1/'
submit_csv_path = '../W/input/google-cloud-ncaa-march-madness-2020-division-1-womens-tournament'
output_path = '../output/W/'

tourney_results = pd.read_csv(os.path.join(path, 'WNCAATourneyDetailedResults.csv'))
seeds = pd.read_csv(os.path.join(path, 'WNCAATourneySeeds.csv'))
regular_results = pd.read_csv(os.path.join(path, 'WRegularSeasonDetailedResults.csv'))
teams = pd.read_csv(os.path.join(path, 'WTeams.csv'))
cities = pd.read_csv(os.path.join(path, 'WGameCities.csv'))
wTeamConferences = pd.read_csv(os.path.join(path, 'WTeamConferences.csv'))

# 3.Make csv file

## Ranking

In [16]:
result_df = tourney_results[tourney_results.DayNum>=132].sort_values(['Season', 'DayNum'], ascending=False)
result_df['place'] = result_df.groupby('Season')['DayNum'].rank(ascending=False, method="first")
result_df.place = result_df.place.astype(int)

winner_df = result_df[result_df.place==1][['Season', 'WTeamID', 'place']]
winner_df.columns.values[1] = 'TeamID'
loser_df = result_df[result_df.place<=63][['Season', 'LTeamID', 'place']]
loser_df.columns.values[1] = 'TeamID'

In [17]:
# best2, 4, 8, 16, 32のリストをSeason分作成
place_list = []

for _ in result_df['Season'].unique():
    place_list.extend([2**i for i in range(1, 7) for _ in range(2**(i-1))])
loser_df['place'] = place_list

In [18]:
ranking_df = pd.concat([winner_df, loser_df], axis=0).sort_values(['Season', 'place']).reset_index(drop=True)
ranking_df.to_csv('../W/intermed/WNCAATourneyRanking.csv', index=False)

In [83]:
# 累積
ranking_df.groupby(['TeamID', 'place']).count().reset_index(drop=False).head()

,TeamID,place,Season
0,3101,64,1
1,3103,64,1
2,3106,64,2
3,3107,32,1
4,3107,64,5


In [87]:
# 直近3年の成績(2013~2019)
sum_3y_df = pd.DataFrame(columns=['TeamID', 'place_1', 'place_2', 'place_4', 'place_8', 'place_16', 'place_32', 'place_64', 'Season'])

for i in range(2013, 2020):
    tmp = ranking_df.loc[(ranking_df.Season==i-1) | (ranking_df.Season == i-2) | (ranking_df.Season == i-3)]
    tmp = tmp.groupby(['TeamID', 'place']).count().reset_index(drop=False)
    tmp = pd.pivot(tmp, index='TeamID', columns='place', values='Season').reset_index().fillna(0)
    tmp.columns = ['TeamID', 'place_1', 'place_2', 'place_4', 'place_8', 'place_16', 'place_32', 'place_64']
    tmp['Season'] = i
    sum_3y_df = pd.concat([sum_3y_df, tmp], axis=0)
sum_3y_df['count'] = sum_3y_df['place_1'] + sum_3y_df['place_2'] + sum_3y_df['place_4'] + sum_3y_df['place_8'] + sum_3y_df['place_16'] + sum_3y_df['place_32'] + sum_3y_df['place_64']
sum_3y_df.head()

,TeamID,place_1,place_2,place_4,place_8,place_16,place_32,place_64,Season,count
0,3107,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013,1.0
1,3113,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013,1.0
2,3114,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2013,3.0
3,3116,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2013,1.0
4,3122,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013,1.0


In [ ]:
# 今後やりたいこと
# 監督の実績とシーズンの途中で交代したか、カンファレンスごとにまとめる、レーティング(MasseyOrdinals)
# 去年出てた選手がどれだけ出ているか
# リーグごとにスコアとかまとめる

### Rnking by Conference

In [4]:
rankng_df = pd.read_csv('../W/intermed/WNCAATourneyRanking.csv')
rankng_df.head()

,Season,TeamID,place
0,2010,3163,1
1,2010,3390,2
2,2010,3124,4
3,2010,3328,4
4,2010,3199,8


In [5]:
wTeamConferences.head()

,Season,TeamID,ConfAbbrev
0,1998,3102,wac
1,1998,3103,mac
2,1998,3104,sec
3,1998,3106,swac
4,1998,3108,swac


In [7]:
join_df = pd.merge(rankng_df, wTeamConferences, on=['Season', 'TeamID'], how='left')
join_df.head()

,Season,TeamID,place,ConfAbbrev
0,2010,3163,1,big_east
1,2010,3390,2,pac_ten
2,2010,3124,4,big_twelve
3,2010,3328,4,big_twelve
4,2010,3199,8,acc


## Team by Conference 失敗したああ

In [48]:
teams = pd.read_csv(os.path.join(path, 'WTeams.csv'))
len(teams)

365

In [3]:
match_df = regular_results[['Season', 'WTeamID', 'LTeamID']]
match_df.columns = ['Season', 'T1_TeamID', 'T2_TeamID']

swap_df = match_df.copy()[['Season', 'T2_TeamID', 'T1_TeamID']]
swap_df.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
match_df = pd.concat([match_df, swap_df], axis=0).drop_duplicates().reset_index(drop=True)

In [49]:
match_dict = {}

for year in match_df['Season'].unique():
    match_dict[year] = {}
    df = match_df[match_df.Season==year]
    for teamID in df['T1_TeamID'].unique():
        match_dict[year][teamID] = set(df[df.T1_TeamID==teamID]['T2_TeamID'])

In [41]:
key_num = 0
conference_dict = {}

for year in match_dict.keys():
    conference_dict[year] = {}
    for x in match_dict[year].keys():
        tmp = list(match_dict[year][x])
        tmp.append(x)
        for y in match_dict[year].keys():
            if x!=y:
                if len(set(tmp).intersection(match_dict[year][y]))>0:
                    tmp.extend(list(match_dict[year][y]))
        conference = set(tmp)
        same_flg = 0
        for z in conference_dict[year].keys():
            if conference==conference_dict[year][z]:
                same_flg=1
        if same_flg==0:
            key_num += 1
            conference_dict[year][key_num] = conference